<a href="https://colab.research.google.com/github/ib-nosirov/CSCI499F21_src/blob/main/UnrolledNeuralNetworks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [106]:
import scipy.io
import numpy as np
from matplotlib import pyplot as plt
import math
import torch
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset, DataLoader
import torch.optim as optim

In [107]:
og_img = scipy.io.loadmat('data/test.m')['im']
plt.figure()
plt.imshow(og_img, interpolation='nearest', cmap='gray')

FileNotFoundError: ignored

In [78]:

class EtaLayer(nn.Module):
  """Custom unrolled layer."""
  def __init__(self, A):
    super().__init__()
    self.A = A
    self.A_T = A.transpose(1, 0)
    self.eta = nn.Parameter(torch.randn(1))

  def forward(self, x, y):
    # y has size n_batch x n_measurements
    x = x.permute(1,0)
    eta = torch.clamp(self.eta, min=0, max=100)
    Ax = self.A.matmul(x)
    y = y.permute(1,0)
    # size n_measurements x n_batch
    res = Ax - y
    x_out = x - eta * self.A_T.matmul(res)
    return x_out.permute(1, 0)


In [79]:
class WLayer(nn.Module):
  """Custom unrolled layer."""
  def __init__(self, A):
    super().__init__()
    self.A = A
    self.A_T = A.transpose(1, 0)
    self.W = nn.Parameter(torch.rand(self.A.size(dim=1),
                                     self.A.size(dim=1)))

  def forward(self, x, y):
    # y has size n_batch x n_measurements
    x = x.permute(1,0)
    W = torch.clamp(self.W, min=0, max=1)
    Ax = self.A.matmul(x)
    y = y.permute(1,0)
    # size n_measurements x n_batch
    res = Ax - y
    x_out = x - W.matmul(self.A_T.matmul(res))
    return x_out.permute(1, 0)

In [100]:
class ExtraNNLayer(nn.Module):
  """Custom unrolled layer."""
  def __init__(self, A):
    super().__init__()
    self.A = A
    self.A_T = A.transpose(1, 0)
    self.eta = nn.Parameter(torch.randn(1))
    self.W = nn.Parameter(torch.rand(self.A.size(dim=1),
                                     self.A.size(dim=1)))
    self.b = nn.Parameter(torch.rand(self.A.size(dim=1), 1))

  def forward(self, x, y):
    x = x.permute(1,0)
    eta = torch.clamp(self.eta, min=0, max=100)
    Ax = self.A.matmul(x)
    y = y.permute(1,0)
    Wx = self.W.matmul(x)
    b = self.b.permute(1,0)
    sigmoid = nn.Sigmoid()
    # size n_measurements x n_batch
    res = Ax - y
    perceptron = Wx + b
    x_out = x - eta * self.A_T.matmul(res) + sigmoid(perceptron)
    return x_out.permute(1, 0)


In [101]:
#import eta_layer 
#import w_layer

class ModelBase(nn.Module):
  def __init__(self):
    super().__init__()

  def EtaLayer(self, A):
    return EtaLayer(A)

  def WLayer(self, A):
    return WLayer(A)

  def ExtraNNLayer(self, A):
    return ExtraNNLayer(A)
    
  def writeTrainingSummaries(self, training_cost):
    return
    #writeToFile


In [102]:
#import model_base
class ModelFc(ModelBase):
  """A fully-connected network with user-specifiable hyperparameters."""

  def __init__(self, A):
    super().__init__()
    #self.x_k1 = super().EtaLayer(A)
    #self.x_k1 = super().WLayer(A)
    self.x_k1 = super().ExtraNNLayer(A)

    # Build the network layer by layer
  def forward(self, x, y):
    x_predict = self.x_k1(x, y)
    for i in range(3):
      x_predict = self.x_k1(x_predict, y) 
    return x_predict

In [103]:
"""A function that trains a neural network."""

class Trainer(nn.Module):
  """Class for training and validating neural network."""
  def __init__(self, data_train, A, max_epochs, batch_size):
    super(Trainer, self).__init__()
    self.train_set = DataLoader(data_train, batch_size, shuffle=True)
#    self.validate_set = DataLoader(data_validate, batch_size, shuffle=True)
    self.A = A
    self.max_epochs = max_epochs
    self.batch_size = batch_size
    self.model = None
    self.loss = None
    self.optim = None

  def train_init(self):
    self.model = ModelFc(self.A)
    self.loss = torch.nn.MSELoss() 
    self.optim = torch.optim.SGD(self.model.parameters(), lr=1e-4)

  def learn(self, loss_val):
    self.optim.zero_grad()
    loss_val.backward()
    self.optim.step()

  def createTrainCost(self):
    running_loss = 0
    for x, y in self.train_set:
      x0 = torch.randn(x.shape[0], x.shape[1])
      x_pred = self.model(x0, y)
      loss_val = self.loss(x_pred, x)
      running_loss += loss_val
      self.learn(loss_val)
    return running_loss / len(self.train_set)

  def train(self):
    self.train_init()
    for epoch in range(self.max_epochs):
      train_cost = self.createTrainCost()
      #validate_cost = self.createValidateCost()
      self.model.writeTrainingSummaries(train_cost)
  
      if(epoch % 10 == 0):
        print('epoch = #', epoch, ', train_cost = ', train_cost)


In [104]:
#A_contents = scipy.io.loadmat('A.mat')
#A = torch.from_numpy(A_contents['A']).float()
A = torch.randn(10, 5)
n_features = A.shape[0]
n_measurements = A.shape[1]
n_samples = 1000
max_epochs = 100
batch_size = 10

A = torch.randn(n_measurements, n_features)
x_train = torch.randn(n_samples, n_features)
y_train = A.matmul(x_train.permute(1,0)).permute(1,0)

data_train = TensorDataset(x_train, y_train)
trainerClass = Trainer(data_train, A, max_epochs, batch_size)
trainerClass.train()

epoch = # 0 , train_cost =  tensor(636871.2500, grad_fn=<DivBackward0>)
epoch = # 10 , train_cost =  tensor(10.2614, grad_fn=<DivBackward0>)
epoch = # 20 , train_cost =  tensor(10.4857, grad_fn=<DivBackward0>)
epoch = # 30 , train_cost =  tensor(10.6864, grad_fn=<DivBackward0>)
epoch = # 40 , train_cost =  tensor(10.6545, grad_fn=<DivBackward0>)
epoch = # 50 , train_cost =  tensor(10.4294, grad_fn=<DivBackward0>)
epoch = # 60 , train_cost =  tensor(10.2057, grad_fn=<DivBackward0>)
epoch = # 70 , train_cost =  tensor(10.7205, grad_fn=<DivBackward0>)
epoch = # 80 , train_cost =  tensor(10.3189, grad_fn=<DivBackward0>)
epoch = # 90 , train_cost =  tensor(10.4517, grad_fn=<DivBackward0>)
